# Analysis of openings gaps in securities.

The following script was written by request of an acquaintance of mine who is into trading and intra-day trading in particular.
The reason for it being written is that this day-trader had observed several "gaps" in leading stock indicies, and wanted to get a more quantitative and analytical understanding of these types of price action. The script is written purely by myself, and might contain inconvenient lines of code, that can be written in a more clean and proper manner, however it gets the job done in the end.

A "gap" is in this project refers to when the price of a security (in this case a stock or stock-index) opens more then 0.5% higher or less than -0.5% lower than the close of the previous trading day.

The object of this script is therefore to provide a basic analysis of these gaps by the following steps:

Step 1) - Detect gaps. 

Step 2) - Identify whether the gap is closed within the day, and if that is the case, how many hours does it take for it to close.

Step 3 - Group gaps based on magnitude/size and number of hours it takes to close the gap.

Step 4) - Displaying all the above in a cross tabulation, as a way of displaying the findings in an easy-to-interpret manner.

#### Initial data import and "manipulation"

The following lines of codes imports the data and packages needed, as well as altering the data such that it can be used in the intended way.

In [5]:
#Importing packages
import pandas as pd 
import yfinance as yf  
import numpy as np
import datetime as dt 
from datetime import datetime,date
from dateutil.relativedelta import relativedelta

In [6]:
#Import data from Yahoo Finance API
ticker='^GDAXI' #THIS IS CAN BE CHANGED TO ANY TICKER, HOWEVER IT MUST BE TRADING WITHIN THE EUROPEAN TRADING HOURS
sd=date.today()-relativedelta(days=729)
ed=date.today()
df=yf.download(tickers=ticker,start=sd,end=ed,interval='1h')

[*********************100%***********************]  1 of 1 completed


In [7]:
#Alters the imported data and index so its ready to be used, this includes change the format of some of the data, as well as adding the prior trading periods close to the given trading period.
df['prior_close']=df['Close'].shift(+1)
df['date_index']=df.index
df['time']=df.index
df['time']=df['time'].dt.time
df['time']=df['time'].astype('str')
df['date']=df.index
df['date']=df['date'].dt.date
df['gap'] = df['Open']/df['prior_close']-1

In [8]:
#The following is a bit of creative manipulation, that acted as necesarry precursor for the later applied for-loop.
#A certain degree of creativity was needed, since a difficulty of the project, was that I was working within two timeframes at the same time.
#In that I was dealing with both the daily close (used to define the opening gap), as well as the hourly quotes, that was needed to calculate the time it took for the gap to be closed.
prior_close=df[df['time']=='09:00:00']
prior_close=prior_close[['date','prior_close','gap']]
prior_close['date']=prior_close['date'].astype('str')
df['date']=df['date'].astype('str')
unique_date=prior_close['date'].unique()
unique_date=unique_date.tolist()

#### Step 1 & 2 - Identifying and categorising gaps.

In [9]:
#The below for-loop in where the main part of the script does its work.
#Lines below are commented with the number relating to which step it is involved in solving (cf. introduction)

filter_df=pd.DataFrame()
tf_df=pd.DataFrame()
tf_list=[]
for i in unique_date: #The for-loop is interated through the list of unique dates as, once again, a creative way of dealing with the "issue" of working with two time-frames simultaneously. 
    #Step 1) - Detect Gaps
    #Gets the closing price of the last day and adds them to all of the hourly-qoutes on that particular day
    cl=prior_close.loc[prior_close['date']==i,'prior_close'] [0]
    tf=df[df['date']==i]
    tf['last_day_close']=cl

    #Calculates the percentage of the gap, and adds them to all of the hourly-qoutes on that particular day
    return_l=prior_close.loc[prior_close['date']==i,'gap'][0]
    rf=df[df['date']==i]
    rf['opening_gap']=return_l
    tf['opening_gap']=rf['opening_gap']

    #Making a sub-dataframe with price has made a gap up (i.e. opening at least 0.5% higher than the close of the previous day)
    gapup_tf=tf[tf['opening_gap']>0.005]
    gapup_tf['close_gap_y/n']=np.where(gapup_tf['Low']<=gapup_tf['last_day_close'],1,0)

    #Making a sub-dataframe with price has made a gap down
    gapdown_tf=tf[tf['opening_gap']<-0.005]
    gapdown_tf['close_gap_y/n']=np.where(gapdown_tf['High']>=gapdown_tf['last_day_close'],1,0)

    tf=pd.concat([gapup_tf,gapdown_tf])

    #Step 2) - Identify whether the gap is closed within the day, and if that is the case, how many hours does it take for it to close.

    #Once again a bit of creative coding, that might have a more clean and "proper" alternative, but it works as intended.
    #All the below lines essentially filters and sorts through the data to determine when, and if, the gap is closed.
    total=tf['close_gap_y/n'].sum() 
    tf['close_within_day']=total

    filter_2=tf[tf['close_within_day']<1]
    filter_2=filter_2[filter_2['close_gap_y/n']==0]
    try:
        filter_2=filter_2.iloc[0]
    except:
        filter_2=filter_2[filter_2['close_gap_y/n']==0]

    filter_1=tf[tf['close_gap_y/n']==1]
    try:
        filter_1=filter_1.iloc[0]
    except:
            filter_1=tf[tf['close_gap_y/n']==1]
    filter_df=filter_df.append(filter_1,ignore_index=True)
    filter_df=filter_df.append(filter_2,ignore_index=True)


#Finally the data obtained from above is stored into a new dataframe.
#And since we have now obtained the amount of hours it takes for the gap to close, if it closes at all that is, we can move to solely working within with daily timeframe.
df=filter_df
df=df.set_index('date_index')
df=df[['High','Low','Open','last_day_close','opening_gap','close_gap_y/n','time','date']]

/Users/anderslindstrom/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/anderslindstrom/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/anderslindstrom/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

#### Step 3 - Grouping of gaps.

In [10]:
#Step 3 -  Group gaps based on magnitude/size and number of hours it takes to close the gap.
#The following sets up conditions and values to be assigned to the values in the dataframe for easier interpretation 
conditions=[
    (df['close_gap_y/n'] ==1)&(df['time']=='09:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='10:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='11:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='12:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='13:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='14:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='15:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='16:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='17:00:00'),
    (df['close_gap_y/n'] ==0)
    ]
values=['1h_close','2h_close','3h_close','4h_close','5h_close','6h_close','7h_close','8h_close','9h_close','no_close']

conditions2=[
    #gap_up
    (df['opening_gap'] >=0.005)&(df['opening_gap']<0.01),
    (df['opening_gap'] >=0.01)&(df['opening_gap']<0.015),
    (df['opening_gap'] >=0.015)&(df['opening_gap']<0.02),
    (df['opening_gap'] >=0.02),
    #gap_down
    (df['opening_gap'] <=-0.005)&(df['opening_gap']>-0.01),
    (df['opening_gap'] <=-0.01)&(df['opening_gap']>-0.015),
    (df['opening_gap'] <=-0.015)&(df['opening_gap']>-0.02),
    (df['opening_gap'] <=0.02),
    ]
values2=['0.5/1%','1/1.5%','1.5/2%','>2%','-0.5/-1%','-1/-1.5%','-1.5/-2%','<-2%']

df['closing_time']=np.select(conditions,values)
df['gap_cat']=np.select(conditions2,values2)


#### Step 4 - Displaying results in a cross tabulation.

In [11]:
#Step 4 - Displaying all the above in a cross tabulation, as a way of displaying the findings in an easy-to-interpret manner.
#This is done by creating a cross tabulation with plotting the gap-sizes against their respective closing time.
#The nunmber of hours it takes for a gap to be closed is then changed to a cummulative tabulation, so that the interpreation can be rephrased as e.g "percentage that closed within 4 hours"
cross_tab=pd.crosstab(df['gap_cat'],df['closing_time'])
cross_tab=pd.DataFrame(cross_tab)

cross_tab['sum']=cross_tab.sum(axis=1)
cross_tab['1h_close']=cross_tab['1h_close']/cross_tab['sum']
cross_tab['2h_close']=(cross_tab['2h_close']/cross_tab['sum'])+cross_tab['1h_close']
cross_tab['3h_close']=(cross_tab['3h_close']/cross_tab['sum'])+cross_tab['2h_close']
cross_tab['4h_close']=(cross_tab['4h_close']/cross_tab['sum'])+cross_tab['3h_close']
cross_tab['5h_close']=(cross_tab['5h_close']/cross_tab['sum'])+cross_tab['4h_close']
cross_tab['6h_close']=(cross_tab['6h_close']/cross_tab['sum'])+cross_tab['5h_close']
cross_tab['7h_close']=(cross_tab['7h_close']/cross_tab['sum'])+cross_tab['6h_close']
cross_tab['8h_close']=(cross_tab['8h_close']/cross_tab['sum'])+cross_tab['7h_close']
cross_tab['9h_close']=(cross_tab['9h_close']/cross_tab['sum'])+cross_tab['8h_close']
cross_tab['no_close']=(cross_tab['no_close']/cross_tab['sum'])

cross_tab=cross_tab.reindex(['<-2%','-1.5/-2%','-1/-1.5%','-0.5/-1%','0.5/1%','1/1.5%','1.5/2%','>2%'])


In [12]:
cross_tab

closing_time,1h_close,2h_close,3h_close,4h_close,5h_close,6h_close,7h_close,8h_close,9h_close,no_close,sum
gap_cat,,,,,,,,,,,
<-2%,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.230769,0.307692,0.307692,0.692308,13
-1.5/-2%,0.000000,0.090909,0.090909,0.090909,0.181818,0.181818,0.181818,0.181818,0.181818,0.818182,11
-1/-1.5%,0.058824,0.088235,0.088235,0.147059,0.176471,0.205882,0.264706,0.323529,0.352941,0.647059,34
-0.5/-1%,0.125000,0.275000,0.350000,0.350000,0.350000,0.400000,0.425000,0.425000,0.425000,0.575000,40
0.5/1%,0.103896,0.142857,0.194805,0.220779,0.220779,0.285714,0.350649,0.402597,0.402597,0.597403,77
1/1.5%,0.055556,0.111111,0.138889,0.166667,0.166667,0.166667,0.194444,0.250000,0.277778,0.722222,36
1.5/2%,0.000000,0.000000,0.000000,0.000000,0.142857,0.285714,0.285714,0.285714,0.285714,0.714286,7
>2%,0.000000,0.000000,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.916667,12


The above cross tab shows the final result of the script.
The rows and column values are to be interpreted as(examples - of course dependent of the data imported):

"26.5% of the gaps between -1% and -1.5% closed within 7 hours." - for values in columns 1-9

"71.4% of the gaps between 1.5% and 2% did not close within the day." - for values in column 10

"In the given period, there was a total of 36 gaps with a size fo 1% - 1.5%" - for values in column 11

